In [4]:
import sqlite3

conn = sqlite3.connect('example.db')
cursor = conn.cursor()

# 테이블 생성
# TABLE IF NOT EXISTS users: 유저가 있지 않으면
# 컬럼명은 id, name, age로 편성하고
# id는 자동으로 증가되는 숫자값
# name은 비울수 없는 값
# age는 숫자값
cursor.execute(
    '''
    CREATE TABLE IF NOT EXISTS users (
      id INTEGER PRIMARY KEY,
      name TEXT NOT NULL,
      age INTEGER
    )
    '''
)



In [5]:
# 데이터 삽입
cursor.execute('INSERT INTO users (name,age) VALUES ("홍길동",20)')

cursor.execute('INSERT INTO users (name,age) VALUES ("김철수",30)')

cursor.execute('INSERT INTO users (name,age) VALUES ("김숙희",30)')
# 삽입된 내용 db에 반영
conn.commit()

In [6]:
# 삽입된 데이터 확인

# cursor.execute는 SQL 구문을 실행하는 것이고
# 구문을 실행한 것을 보여주진 않습니다.
cursor.execute('SELECT * FROM users')
# 결과를 보기위해서는 fetch를 해야 합니다.
# 다 가져오는 것은 fetchall이고 하나만 가져오는 것은 fetchone 입니다.
print(cursor.fetchall())


[(1, '홍길동', 20), (2, '김철수', 30), (3, '김숙희', 30)]


In [7]:
# 데이터 수정
cursor.execute('UPDATE users SET age = ? WHERE name = ?',(25,'김철수') )
conn.commit()

In [8]:
cursor.execute('SELECT * FROM users')
print(cursor.fetchall())

[(1, '홍길동', 20), (2, '김철수', 25), (3, '김숙희', 30)]


In [11]:
# 데이터 삭제
cursor.execute('DELETE FROM users WHERE name = ?',('김철수',))
conn.commit()

In [12]:
cursor.execute('SELECT * FROM users')
print(cursor.fetchall())

[(1, '홍길동', 20), (3, '김숙희', 30)]


In [13]:
# db연결해제
conn.close()

# SQLite3 CRUD 연습문제

## 🎯 문제 1: 학생 관리 시스템

### 1단계: 테이블 만들기
- `school.db` 데이터베이스에 연결하세요
- `students` 테이블을 만드세요
  - id: 기본키 (숫자)
  - name: 이름 (텍스트)
  - age: 나이 (숫자)
  - grade: 학년 (텍스트)

### 2단계: 학생 추가하기 (CREATE)
다음 학생들을 데이터베이스에 추가하세요:
- 김철수, 20살, 3학년
- 이영희, 19살, 2학년  
- 박민수, 21살, 4학년

### 3단계: 학생 조회하기 (READ)
- 모든 학생의 정보를 조회해서 출력하세요
- 20살 이상인 학생만 조회해서 "이름: ○○, 나이: ○○" 형태로 출력하세요

### 4단계: 정보 수정하기 (UPDATE)
- 김철수의 나이를 22살로 수정하세요
- 수정 후 김철수의 정보를 조회해서 확인하세요

### 5단계: 학생 삭제하기 (DELETE)
- 박민수 학생을 삭제하세요
- 삭제 후 남은 모든 학생을 조회해서 확인하세요

In [14]:
# 1단계
print("\n1단계: 테이블 만들기")
conn = sqlite3.connect('school.db')
cursor = conn.cursor()

cursor.execute('''
    CREATE TABLE IF NOT EXISTS students (
        id INTEGER PRIMARY KEY,
        name TEXT,
        age INTEGER,
        grade TEXT
    )
''')
print("테이블 생성 완료!")


1단계: 테이블 만들기
테이블 생성 완료!


In [15]:
# 2단계
print("\n2단계: 학생 추가하기")
cursor.execute("INSERT INTO students (id, name, age, grade) VALUES (1, '김철수', 20, '3학년')")
cursor.execute("INSERT INTO students (id, name, age, grade) VALUES (2, '이영희', 19, '2학년')")
cursor.execute("INSERT INTO students (id, name, age, grade) VALUES (3, '박민수', 21, '4학년')")
conn.commit()
print("학생 추가 완료!")


2단계: 학생 추가하기
학생 추가 완료!


In [18]:
# 3단계
print("\n3단계: 학생 조회하기")

cursor.execute('SELECT * FROM students')
students = cursor.fetchall()

# 전체학생
for student in students:
  print(student)

# 20살 이상 학생

cursor.execute('SELECT * FROM students WHERE age >= 20')
older_students = cursor.fetchall()

for student in older_students:
  print(f"이름: {student[1]}, 나이: {student[2]}")



3단계: 학생 조회하기
(1, '김철수', 20, '3학년')
(2, '이영희', 19, '2학년')
(3, '박민수', 21, '4학년')
이름: 김철수, 나이: 20
이름: 박민수, 나이: 21


In [19]:
# 4단계
print("\n4단계: 정보 수정하기")
cursor.execute("UPDATE students SET age = 20 WHERE name = '김철수'")
conn.commit()

cursor.execute("SELECT * FROM students WHERE name = '김철수'")
result = cursor.fetchone()
print(f"수정된 김철수 정보: {result}")



4단계: 정보 수정하기
수정된 김철수 정보: (1, '김철수', 20, '3학년')


In [20]:
# 5단계
print("\n5단계: 학생 삭제하기")
cursor.execute("DELETE FROM students WHERE name= '박민수'")
conn.commit()

cursor.execute("SELECT * FROM students")
remaining = cursor.fetchall()

for student in remaining:
  print(student)

conn.close()


5단계: 학생 삭제하기
(1, '김철수', 20, '3학년')
(2, '이영희', 19, '2학년')


# SQLAlchemy colab 실습

In [21]:
# 라이브러리 임포트
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

In [22]:
# 데이터베이스 연결 설정
# 'sqlite:///a/b/c/sample.db'와 'sqlite:///./a/b/c/sample.db'는 같습니다.
# echo=True를 하게 되면 SQL문이 실행이 될 때마다 출력창에서
# 반복해 출력해줍니다. 공부할 때 좋습니다.

engine = create_engine('sqlite:///sample.db',echo=True)

In [23]:
# DB와 연동되는 모델을 생성하기 위한 기본 모델
Base = declarative_base()

/tmp/ipython-input-1027333092.py:2: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [24]:
class User(Base):
  __tablename__ = 'users' # 테이블 이름

  id = Column(Integer,primary_key=True) # 기본키
  name = Column(String(50)) # 이름 컬럼, 50자 제한
  age = Column(Integer)

  def __repr__(self):
    return f'<{self.id}, {self.name}, {self.age}>'


In [26]:
Base.metadata.create_all(engine) # 테이블 생성

2025-09-18 07:14:19,647 INFO sqlalchemy.engine.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.Engine:BEGIN (implicit)


2025-09-18 07:14:19,651 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("users")


INFO:sqlalchemy.engine.Engine:PRAGMA main.table_info("users")


2025-09-18 07:14:19,652 INFO sqlalchemy.engine.Engine [raw sql] ()


...

INFO:sqlalchemy.engine.Engine:
CREATE TABLE users (
	id INTEGER NOT NULL, 
	name VARCHAR(50), 
	age INTEGER, 
	PRIMARY KEY (id)
)




2025-09-18 07:14:19,668 INFO sqlalchemy.engine.Engine [no key 0.00513s] ()


INFO:sqlalchemy.engine.Engine:[no key 0.00513s] ()


2025-09-18 07:14:19,689 INFO sqlalchemy.engine.Engine COMMIT


INFO:sqlalchemy.engine.Engine:COMMIT


In [27]:
# 세션 생성(DB와 연결하는 코드)
Session = sessionmaker(bind=engine)

session = Session()

In [28]:
# 데이터삽입(Insert)

new_user = User(name="licat",age=10)

session.add(new_user)
session.commit()

2025-09-18 07:22:12,371 INFO sqlalchemy.engine.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.Engine:BEGIN (implicit)


2025-09-18 07:22:12,375 INFO sqlalchemy.engine.Engine INSERT INTO users (name, age) VALUES (?, ?)


INFO:sqlalchemy.engine.Engine:INSERT INTO users (name, age) VALUES (?, ?)


2025-09-18 07:22:12,378 INFO sqlalchemy.engine.Engine [generated in 0.00237s] ('licat', 10)


INFO:sqlalchemy.engine.Engine:[generated in 0.00237s] ('licat', 10)


2025-09-18 07:22:12,382 INFO sqlalchemy.engine.Engine COMMIT


INFO:sqlalchemy.engine.Engine:COMMIT


In [29]:
# 데이터읽기(Read)

session.query(User).all()

2025-09-18 07:23:51,691 INFO sqlalchemy.engine.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.Engine:BEGIN (implicit)


2025-09-18 07:23:51,696 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.age AS users_age 
FROM users


INFO:sqlalchemy.engine.Engine:SELECT users.id AS users_id, users.name AS users_name, users.age AS users_age 
FROM users


2025-09-18 07:23:51,698 INFO sqlalchemy.engine.Engine [generated in 0.00241s] ()


INFO:sqlalchemy.engine.Engine:[generated in 0.00241s] ()


[<1, licat, 10>]

In [34]:
# 데이터읽기 (WHERE)
session.query(User).filter(User.name == 'licat').first()

2025-09-18 07:34:06,202 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.age AS users_age 
FROM users 
WHERE users.name = ?
 LIMIT ? OFFSET ?


INFO:sqlalchemy.engine.Engine:SELECT users.id AS users_id, users.name AS users_name, users.age AS users_age 
FROM users 
WHERE users.name = ?
 LIMIT ? OFFSET ?


2025-09-18 07:34:06,206 INFO sqlalchemy.engine.Engine [cached since 82.42s ago] ('licat', 1, 0)


INFO:sqlalchemy.engine.Engine:[cached since 82.42s ago] ('licat', 1, 0)


<1, licat, 10>

In [35]:
# 데이터수정 (update)

licat = session.query(User).filter(User.name == 'licat').first()

if licat:
  licat.age = 100
  session.commit()

session.query(User).all()


2025-09-18 07:36:30,997 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.age AS users_age 
FROM users 
WHERE users.name = ?
 LIMIT ? OFFSET ?


INFO:sqlalchemy.engine.Engine:SELECT users.id AS users_id, users.name AS users_name, users.age AS users_age 
FROM users 
WHERE users.name = ?
 LIMIT ? OFFSET ?


2025-09-18 07:36:31,000 INFO sqlalchemy.engine.Engine [cached since 227.2s ago] ('licat', 1, 0)


INFO:sqlalchemy.engine.Engine:[cached since 227.2s ago] ('licat', 1, 0)


2025-09-18 07:36:31,004 INFO sqlalchemy.engine.Engine UPDATE users SET age=? WHERE users.id = ?


...

2025-09-18 07:36:31,032 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.age AS users_age 
FROM users


INFO:sqlalchemy.engine.Engine:SELECT users.id AS users_id, users.name AS users_name, users.age AS users_age 
FROM users


2025-09-18 07:36:31,034 INFO sqlalchemy.engine.Engine [cached since 759.3s ago] ()


INFO:sqlalchemy.engine.Engine:[cached since 759.3s ago] ()


[<1, licat, 100>]

In [36]:
# 데이터삭제 (delete)

if licat:
  session.delete(licat)
  session.commit()

session.query(User).all()

2025-09-18 07:39:21,850 INFO sqlalchemy.engine.Engine DELETE FROM users WHERE users.id = ?


INFO:sqlalchemy.engine.Engine:DELETE FROM users WHERE users.id = ?


2025-09-18 07:39:21,853 INFO sqlalchemy.engine.Engine [generated in 0.00263s] (1,)


INFO:sqlalchemy.engine.Engine:[generated in 0.00263s] (1,)


2025-09-18 07:39:21,859 INFO sqlalchemy.engine.Engine COMMIT


...

2025-09-18 07:39:21,874 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.age AS users_age 
FROM users


INFO:sqlalchemy.engine.Engine:SELECT users.id AS users_id, users.name AS users_name, users.age AS users_age 
FROM users


2025-09-18 07:39:21,876 INFO sqlalchemy.engine.Engine [cached since 930.2s ago] ()


INFO:sqlalchemy.engine.Engine:[cached since 930.2s ago] ()


[]

In [37]:
# 연결해제
session.close()

2025-09-18 07:40:00,532 INFO sqlalchemy.engine.Engine ROLLBACK


INFO:sqlalchemy.engine.Engine:ROLLBACK
